In [1]:
import numpy as np
import pandas as pd
import xlrd
import os
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV,\
cross_val_score, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier,\
ExtraTreesClassifier, VotingClassifier, StackingRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder,\
OrdinalEncoder, MaxAbsScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, f1_score,\
accuracy_score, classification_report, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from xgboost import XGBClassifier
import xgboost as xgb
from matplotlib import pyplot as plt
import warnings

In [2]:
season_df = pd.read_csv('/Users/aheinke/Documents/Flatiron/NYC-DS-010923/Phase_3/Phase3_Proj/PROJ_CSVs/WRegularSeasonDetailedResults.csv')
season_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,11,3103,63,3237,49,H,0,23,54,...,13,6,10,11,27,11,23,7,6,19
1,2010,11,3104,73,3399,68,N,0,26,62,...,21,14,27,14,26,7,20,4,2,27
2,2010,11,3110,71,3224,59,A,0,29,62,...,14,19,23,17,23,8,15,6,0,15
3,2010,11,3111,63,3267,58,A,0,27,52,...,26,16,25,22,22,15,11,14,5,14
4,2010,11,3119,74,3447,70,H,1,30,74,...,17,11,21,21,32,12,14,4,2,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70002,2023,113,3268,96,3234,68,H,0,34,71,...,38,11,15,18,19,14,24,5,3,14
70003,2023,113,3385,69,3163,64,A,0,23,63,...,21,12,15,11,25,14,12,5,7,19
70004,2023,113,3416,57,3396,53,H,0,19,60,...,26,14,17,16,19,5,9,10,9,20
70005,2023,113,3437,67,3177,64,H,0,24,67,...,19,8,11,21,26,13,12,9,2,14


In [3]:
tourney_df = pd.read_csv('/Users/aheinke/Documents/Flatiron/NYC-DS-010923/Phase_3/Phase3_Proj/PROJ_CSVs/WNCAATourneyDetailedResults.csv')
tourney_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,138,3124,69,3201,55,N,0,28,57,...,34,3,5,17,19,12,18,4,1,18
1,2010,138,3173,67,3395,66,N,0,23,59,...,27,14,15,18,26,8,8,8,6,22
2,2010,138,3181,72,3214,37,H,0,26,57,...,15,3,8,10,21,4,16,6,4,20
3,2010,138,3199,75,3256,61,H,0,25,63,...,20,17,22,16,21,13,16,5,4,24
4,2010,138,3207,62,3265,42,N,0,24,68,...,26,11,17,16,22,9,10,3,4,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2022,147,3163,91,3301,87,N,2,37,77,...,23,16,19,6,30,20,13,4,7,16
756,2022,147,3257,62,3276,50,N,0,25,58,...,14,15,20,10,24,9,21,6,2,12
757,2022,151,3163,63,3390,58,N,0,21,57,...,23,8,13,11,23,10,11,11,3,16
758,2022,151,3376,72,3257,59,N,0,27,57,...,8,4,7,11,18,5,15,13,2,17


In [4]:
merged_df = pd.concat([season_df, tourney_df])

# sort the merged dataframe by "col_name" in ascending order
sorted_df = merged_df.sort_values(['Season', 'DayNum'], ascending=[True, True])

# reset the index
games_df = sorted_df.reset_index(drop=True)
games_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,11,3103,63,3237,49,H,0,23,54,...,13,6,10,11,27,11,23,7,6,19
1,2010,11,3104,73,3399,68,N,0,26,62,...,21,14,27,14,26,7,20,4,2,27
2,2010,11,3110,71,3224,59,A,0,29,62,...,14,19,23,17,23,8,15,6,0,15
3,2010,11,3111,63,3267,58,A,0,27,52,...,26,16,25,22,22,15,11,14,5,14
4,2010,11,3119,74,3447,70,H,1,30,74,...,17,11,21,21,32,12,14,4,2,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,2023,113,3268,96,3234,68,H,0,34,71,...,38,11,15,18,19,14,24,5,3,14
70763,2023,113,3385,69,3163,64,A,0,23,63,...,21,12,15,11,25,14,12,5,7,19
70764,2023,113,3416,57,3396,53,H,0,19,60,...,26,14,17,16,19,5,9,10,9,20
70765,2023,113,3437,67,3177,64,H,0,24,67,...,19,8,11,21,26,13,12,9,2,14


In [5]:
seeds_df = pd.read_csv('/Users/aheinke/Documents/Flatiron/NYC-DS-010923/Phase_3/Phase3_Proj/PROJ_CSVs/WNCAATourneySeeds.csv')
seeds_df

,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163
2,1998,W03,3112
3,1998,W04,3301
4,1998,W05,3272
...,...,...,...
1535,2022,Z12,3125
1536,2022,Z13,3138
1537,2022,Z14,3110
1538,2022,Z15,3218


In [6]:
seeds_df['SeedNum'] = seeds_df['Seed'].str.extract('(\d+)').astype(int)
seeds_df

,Season,Seed,TeamID,SeedNum
0,1998,W01,3330,1
1,1998,W02,3163,2
2,1998,W03,3112,3
3,1998,W04,3301,4
4,1998,W05,3272,5
...,...,...,...,...
1535,2022,Z12,3125,12
1536,2022,Z13,3138,13
1537,2022,Z14,3110,14
1538,2022,Z15,3218,15


In [7]:
team_season_seed_dict = dict(zip(zip(seeds_df['TeamID'], seeds_df['Season']), seeds_df['SeedNum']))
team_season_seed_dict

{(3330, 1998): 1,
 (3163, 1998): 2,
 (3112, 1998): 3,
 (3301, 1998): 4,
 (3272, 1998): 5,
 (3438, 1998): 6,
 (3208, 1998): 7,
 (3307, 1998): 8,
 (3304, 1998): 9,
 (3203, 1998): 10,
 (3374, 1998): 11,
 (3464, 1998): 12,
 (3263, 1998): 13,
 (3365, 1998): 14,
 (3193, 1998): 15,
 (3384, 1998): 16,
 (3403, 1998): 1,
 (3104, 1998): 2,
 (3256, 1998): 3,
 (3345, 1998): 4,
 (3179, 1998): 5,
 (3155, 1998): 6,
 (3417, 1998): 7,
 (3283, 1998): 8,
 (3323, 1998): 9,
 (3276, 1998): 10,
 (3274, 1998): 11,
 (3161, 1998): 12,
 (3449, 1998): 13,
 (3221, 1998): 14,
 (3422, 1998): 15,
 (3212, 1998): 16,
 (3397, 1998): 1,
 (3314, 1998): 2,
 (3228, 1998): 3,
 (3235, 1998): 4,
 (3353, 1998): 5,
 (3435, 1998): 6,
 (3198, 1998): 7,
 (3443, 1998): 8,
 (3372, 1998): 9,
 (3266, 1998): 10,
 (3364, 1998): 11,
 (3332, 1998): 12,
 (3245, 1998): 13,
 (3453, 1998): 14,
 (3224, 1998): 15,
 (3251, 1998): 16,
 (3390, 1998): 1,
 (3181, 1998): 2,
 (3196, 1998): 3,
 (3234, 1998): 4,
 (3242, 1998): 5,
 (3458, 1998): 6,
 (3428,

In [8]:
games_df['WSeed'] = games_df.apply(lambda row: team_season_seed_dict.get((row['WTeamID'], row['Season']),\
                                                                           0), axis=1)

In [9]:
games_df['LSeed'] = games_df.apply(lambda row: team_season_seed_dict.get((row['LTeamID'], row['Season']),\
                                                                           0), axis=1)

In [10]:
games_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,WSeed,LSeed
0,2010,11,3103,63,3237,49,H,0,23,54,...,10,11,27,11,23,7,6,19,0,0
1,2010,11,3104,73,3399,68,N,0,26,62,...,27,14,26,7,20,4,2,27,0,0
2,2010,11,3110,71,3224,59,A,0,29,62,...,23,17,23,8,15,6,0,15,0,0
3,2010,11,3111,63,3267,58,A,0,27,52,...,25,22,22,15,11,14,5,14,0,0
4,2010,11,3119,74,3447,70,H,1,30,74,...,21,21,32,12,14,4,2,14,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,2023,113,3268,96,3234,68,H,0,34,71,...,15,18,19,14,24,5,3,14,0,0
70763,2023,113,3385,69,3163,64,A,0,23,63,...,15,11,25,14,12,5,7,19,0,0
70764,2023,113,3416,57,3396,53,H,0,19,60,...,17,16,19,5,9,10,9,20,0,0
70765,2023,113,3437,67,3177,64,H,0,24,67,...,11,21,26,13,12,9,2,14,0,0


In [11]:
games_df['GameID'] = games_df.index
games_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,WSeed,LSeed,GameID
0,2010,11,3103,63,3237,49,H,0,23,54,...,11,27,11,23,7,6,19,0,0,0
1,2010,11,3104,73,3399,68,N,0,26,62,...,14,26,7,20,4,2,27,0,0,1
2,2010,11,3110,71,3224,59,A,0,29,62,...,17,23,8,15,6,0,15,0,0,2
3,2010,11,3111,63,3267,58,A,0,27,52,...,22,22,15,11,14,5,14,0,0,3
4,2010,11,3119,74,3447,70,H,1,30,74,...,21,32,12,14,4,2,14,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,2023,113,3268,96,3234,68,H,0,34,71,...,18,19,14,24,5,3,14,0,0,70762
70763,2023,113,3385,69,3163,64,A,0,23,63,...,11,25,14,12,5,7,19,0,0,70763
70764,2023,113,3416,57,3396,53,H,0,19,60,...,16,19,5,9,10,9,20,0,0,70764
70765,2023,113,3437,67,3177,64,H,0,24,67,...,21,26,13,12,9,2,14,0,0,70765


In [12]:
cols_to_keep = ['GameID','Season', 'DayNum', 'WTeamID', 'WSeed', 'WScore', 'WLoc', 'NumOT', 'WFGM', 'WFGA', 'WFGM3',\
                'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3',\
               'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']
winners_df = games_df[cols_to_keep]
winners_df

,GameID,Season,DayNum,WTeamID,WSeed,WScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,0,2010,11,3103,0,63,H,0,23,54,...,13,6,10,11,27,11,23,7,6,19
1,1,2010,11,3104,0,73,N,0,26,62,...,21,14,27,14,26,7,20,4,2,27
2,2,2010,11,3110,0,71,A,0,29,62,...,14,19,23,17,23,8,15,6,0,15
3,3,2010,11,3111,0,63,A,0,27,52,...,26,16,25,22,22,15,11,14,5,14
4,4,2010,11,3119,0,74,H,1,30,74,...,17,11,21,21,32,12,14,4,2,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,70762,2023,113,3268,0,96,H,0,34,71,...,38,11,15,18,19,14,24,5,3,14
70763,70763,2023,113,3385,0,69,A,0,23,63,...,21,12,15,11,25,14,12,5,7,19
70764,70764,2023,113,3416,0,57,H,0,19,60,...,26,14,17,16,19,5,9,10,9,20
70765,70765,2023,113,3437,0,67,H,0,24,67,...,19,8,11,21,26,13,12,9,2,14


In [13]:
winners_df['Win1_Lose0'] = 1

<ipython-input-13-e8e23d5c63a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winners_df['Win1_Lose0'] = 1


In [14]:
winners_df

,GameID,Season,DayNum,WTeamID,WSeed,WScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Win1_Lose0
0,0,2010,11,3103,0,63,H,0,23,54,...,6,10,11,27,11,23,7,6,19,1
1,1,2010,11,3104,0,73,N,0,26,62,...,14,27,14,26,7,20,4,2,27,1
2,2,2010,11,3110,0,71,A,0,29,62,...,19,23,17,23,8,15,6,0,15,1
3,3,2010,11,3111,0,63,A,0,27,52,...,16,25,22,22,15,11,14,5,14,1
4,4,2010,11,3119,0,74,H,1,30,74,...,11,21,21,32,12,14,4,2,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,70762,2023,113,3268,0,96,H,0,34,71,...,11,15,18,19,14,24,5,3,14,1
70763,70763,2023,113,3385,0,69,A,0,23,63,...,12,15,11,25,14,12,5,7,19,1
70764,70764,2023,113,3416,0,57,H,0,19,60,...,14,17,16,19,5,9,10,9,20,1
70765,70765,2023,113,3437,0,67,H,0,24,67,...,8,11,21,26,13,12,9,2,14,1


In [15]:
dict = {'WTeamID': 'TeamID', 'WSeed': 'Seed', 'WScore': 'Score'}

In [16]:
winners_df.rename(columns = dict, inplace = True)
winners_df

/Users/aheinke/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,GameID,Season,DayNum,TeamID,Seed,Score,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Win1_Lose0
0,0,2010,11,3103,0,63,H,0,23,54,...,6,10,11,27,11,23,7,6,19,1
1,1,2010,11,3104,0,73,N,0,26,62,...,14,27,14,26,7,20,4,2,27,1
2,2,2010,11,3110,0,71,A,0,29,62,...,19,23,17,23,8,15,6,0,15,1
3,3,2010,11,3111,0,63,A,0,27,52,...,16,25,22,22,15,11,14,5,14,1
4,4,2010,11,3119,0,74,H,1,30,74,...,11,21,21,32,12,14,4,2,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,70762,2023,113,3268,0,96,H,0,34,71,...,11,15,18,19,14,24,5,3,14,1
70763,70763,2023,113,3385,0,69,A,0,23,63,...,12,15,11,25,14,12,5,7,19,1
70764,70764,2023,113,3416,0,57,H,0,19,60,...,14,17,16,19,5,9,10,9,20,1
70765,70765,2023,113,3437,0,67,H,0,24,67,...,8,11,21,26,13,12,9,2,14,1


In [17]:
cols_to_keep = ['GameID','Season', 'DayNum', 'LTeamID', 'LSeed', 'LScore', 'WLoc', 'NumOT', 'WFGM', 'WFGA', 'WFGM3',\
                'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3',\
               'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']
losers_df = games_df[cols_to_keep]
losers_df

,GameID,Season,DayNum,LTeamID,LSeed,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,0,2010,11,3237,0,49,H,0,23,54,...,13,6,10,11,27,11,23,7,6,19
1,1,2010,11,3399,0,68,N,0,26,62,...,21,14,27,14,26,7,20,4,2,27
2,2,2010,11,3224,0,59,A,0,29,62,...,14,19,23,17,23,8,15,6,0,15
3,3,2010,11,3267,0,58,A,0,27,52,...,26,16,25,22,22,15,11,14,5,14
4,4,2010,11,3447,0,70,H,1,30,74,...,17,11,21,21,32,12,14,4,2,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,70762,2023,113,3234,0,68,H,0,34,71,...,38,11,15,18,19,14,24,5,3,14
70763,70763,2023,113,3163,0,64,A,0,23,63,...,21,12,15,11,25,14,12,5,7,19
70764,70764,2023,113,3396,0,53,H,0,19,60,...,26,14,17,16,19,5,9,10,9,20
70765,70765,2023,113,3177,0,64,H,0,24,67,...,19,8,11,21,26,13,12,9,2,14


In [18]:
losers_df['Win1_Lose0'] = 0

<ipython-input-18-7a994eabc160>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  losers_df['Win1_Lose0'] = 0


In [19]:
losers_df

,GameID,Season,DayNum,LTeamID,LSeed,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Win1_Lose0
0,0,2010,11,3237,0,49,H,0,23,54,...,6,10,11,27,11,23,7,6,19,0
1,1,2010,11,3399,0,68,N,0,26,62,...,14,27,14,26,7,20,4,2,27,0
2,2,2010,11,3224,0,59,A,0,29,62,...,19,23,17,23,8,15,6,0,15,0
3,3,2010,11,3267,0,58,A,0,27,52,...,16,25,22,22,15,11,14,5,14,0
4,4,2010,11,3447,0,70,H,1,30,74,...,11,21,21,32,12,14,4,2,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,70762,2023,113,3234,0,68,H,0,34,71,...,11,15,18,19,14,24,5,3,14,0
70763,70763,2023,113,3163,0,64,A,0,23,63,...,12,15,11,25,14,12,5,7,19,0
70764,70764,2023,113,3396,0,53,H,0,19,60,...,14,17,16,19,5,9,10,9,20,0
70765,70765,2023,113,3177,0,64,H,0,24,67,...,8,11,21,26,13,12,9,2,14,0


In [20]:
dict = {'LTeamID': 'TeamID', 'LSeed': 'Seed', 'LScore': 'Score'}

In [21]:
losers_df.rename(columns = dict, inplace = True)
losers_df

/Users/aheinke/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,GameID,Season,DayNum,TeamID,Seed,Score,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Win1_Lose0
0,0,2010,11,3237,0,49,H,0,23,54,...,6,10,11,27,11,23,7,6,19,0
1,1,2010,11,3399,0,68,N,0,26,62,...,14,27,14,26,7,20,4,2,27,0
2,2,2010,11,3224,0,59,A,0,29,62,...,19,23,17,23,8,15,6,0,15,0
3,3,2010,11,3267,0,58,A,0,27,52,...,16,25,22,22,15,11,14,5,14,0
4,4,2010,11,3447,0,70,H,1,30,74,...,11,21,21,32,12,14,4,2,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,70762,2023,113,3234,0,68,H,0,34,71,...,11,15,18,19,14,24,5,3,14,0
70763,70763,2023,113,3163,0,64,A,0,23,63,...,12,15,11,25,14,12,5,7,19,0
70764,70764,2023,113,3396,0,53,H,0,19,60,...,14,17,16,19,5,9,10,9,20,0
70765,70765,2023,113,3177,0,64,H,0,24,67,...,8,11,21,26,13,12,9,2,14,0


In [22]:
merged_df2 = pd.concat([winners_df, losers_df])

# sort the merged dataframe by "col_name" in ascending order
sorted_df2 = merged_df2.sort_values('GameID', ascending = True)

# reset the index
ncaa_wmns_df = sorted_df2.reset_index(drop=True)
ncaa_wmns_df

,GameID,Season,DayNum,TeamID,Seed,Score,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Win1_Lose0
0,0,2010,11,3103,0,63,H,0,23,54,...,6,10,11,27,11,23,7,6,19,1
1,0,2010,11,3237,0,49,H,0,23,54,...,6,10,11,27,11,23,7,6,19,0
2,1,2010,11,3399,0,68,N,0,26,62,...,14,27,14,26,7,20,4,2,27,0
3,1,2010,11,3104,0,73,N,0,26,62,...,14,27,14,26,7,20,4,2,27,1
4,2,2010,11,3110,0,71,A,0,29,62,...,19,23,17,23,8,15,6,0,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141529,70764,2023,113,3416,0,57,H,0,19,60,...,14,17,16,19,5,9,10,9,20,1
141530,70765,2023,113,3177,0,64,H,0,24,67,...,8,11,21,26,13,12,9,2,14,0
141531,70765,2023,113,3437,0,67,H,0,24,67,...,8,11,21,26,13,12,9,2,14,1
141532,70766,2023,113,3466,0,75,A,0,27,62,...,11,21,8,19,6,15,3,3,12,1


In [23]:
ncaa_wmns_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141534 entries, 0 to 141533
Data columns (total 35 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   GameID      141534 non-null  int64 
 1   Season      141534 non-null  int64 
 2   DayNum      141534 non-null  int64 
 3   TeamID      141534 non-null  int64 
 4   Seed        141534 non-null  int64 
 5   Score       141534 non-null  int64 
 6   WLoc        141534 non-null  object
 7   NumOT       141534 non-null  int64 
 8   WFGM        141534 non-null  int64 
 9   WFGA        141534 non-null  int64 
 10  WFGM3       141534 non-null  int64 
 11  WFGA3       141534 non-null  int64 
 12  WFTM        141534 non-null  int64 
 13  WFTA        141534 non-null  int64 
 14  WOR         141534 non-null  int64 
 15  WDR         141534 non-null  int64 
 16  WAst        141534 non-null  int64 
 17  WTO         141534 non-null  int64 
 18  WStl        141534 non-null  int64 
 19  WBlk        141534 non-

In [24]:
wloc_map = {'H': 1, 'A': 2, 'N': 0}
ncaa_wmns_df['WLoc'] = ncaa_wmns_df['WLoc'].map(wloc_map)
ncaa_wmns_df

,GameID,Season,DayNum,TeamID,Seed,Score,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Win1_Lose0
0,0,2010,11,3103,0,63,1,0,23,54,...,6,10,11,27,11,23,7,6,19,1
1,0,2010,11,3237,0,49,1,0,23,54,...,6,10,11,27,11,23,7,6,19,0
2,1,2010,11,3399,0,68,0,0,26,62,...,14,27,14,26,7,20,4,2,27,0
3,1,2010,11,3104,0,73,0,0,26,62,...,14,27,14,26,7,20,4,2,27,1
4,2,2010,11,3110,0,71,2,0,29,62,...,19,23,17,23,8,15,6,0,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141529,70764,2023,113,3416,0,57,1,0,19,60,...,14,17,16,19,5,9,10,9,20,1
141530,70765,2023,113,3177,0,64,1,0,24,67,...,8,11,21,26,13,12,9,2,14,0
141531,70765,2023,113,3437,0,67,1,0,24,67,...,8,11,21,26,13,12,9,2,14,1
141532,70766,2023,113,3466,0,75,2,0,27,62,...,11,21,8,19,6,15,3,3,12,1


In [ ]:
X = ncaa_wmns_df.drop('Win1_Lose0', axis=1)
y = ncaa_wmns_df['Win1_Lose0']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
xgb_model = xgb.XGBClassifier()

In [ ]:
xgb_model.fit(X_train, y_train)

In [ ]:
y_pred = xgb_model.predict(X_test)

In [ ]:
print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

In [25]:
ncaa_wmns_df2 = ncaa_wmns_df.loc[ncaa_wmns_df['Season'] < 2023]
ncaa_wmns_df2

,GameID,Season,DayNum,TeamID,Seed,Score,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Win1_Lose0
0,0,2010,11,3103,0,63,1,0,23,54,...,6,10,11,27,11,23,7,6,19,1
1,0,2010,11,3237,0,49,1,0,23,54,...,6,10,11,27,11,23,7,6,19,0
2,1,2010,11,3399,0,68,0,0,26,62,...,14,27,14,26,7,20,4,2,27,0
3,1,2010,11,3104,0,73,0,0,26,62,...,14,27,14,26,7,20,4,2,27,1
4,2,2010,11,3110,0,71,2,0,29,62,...,19,23,17,23,8,15,6,0,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132333,66166,2022,151,3390,1,58,0,0,21,57,...,8,13,11,23,10,11,11,3,16,0
132334,66167,2022,151,3376,1,72,0,0,27,57,...,4,7,11,18,5,15,13,2,17,1
132335,66167,2022,151,3257,1,59,0,0,27,57,...,4,7,11,18,5,15,13,2,17,0
132336,66168,2022,153,3376,1,64,0,0,22,60,...,1,4,3,16,14,14,4,5,21,1


In [26]:
X = ncaa_wmns_df2.drop(['Win1_Lose0', 'WFGM', 'LFGM'], axis=1)
y = ncaa_wmns_df2['Win1_Lose0']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
xgb_model = xgb.XGBClassifier()

In [28]:
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [29]:
y_pred = xgb_model.predict(X_test)

In [30]:
print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

0.9073976122109717
0.9046621267679413
0.9112769485903814
0.9079574899545608


The outcome of the code represents the evaluation metrics for a trained XGBoost classifier model on the test dataset.

The accuracy score of 0.9073976122109717 indicates the proportion of correctly predicted outcomes out of all predictions made by the model.

The precision score of 0.9046621267679413 indicates the proportion of true positive predictions out of all positive predictions made by the model.

The recall score of 0.9112769485903814 indicates the proportion of true positive predictions out of all actual positive outcomes in the test dataset.

The F1 score of 0.9079574899545608 is the harmonic mean of precision and recall and represents the balance between precision and recall of the model.

In summary, the XGBoost model achieved good performance with high accuracy, precision, recall, and F1 scores on the test dataset, indicating that it can effectively predict the outcomes of NCAA Women's Basketball games.

In [31]:
# Split your data into features and target variable
X = ncaa_wmns_df2.drop('Win1_Lose0', axis=1)
y = ncaa_wmns_df2['Win1_Lose0']

# Define your XGBoost model and parameters
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8
)

# Define your cross-validation strategy
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# Evaluate your model using cross-validation
scores = cross_val_score(model, X, y, cv=kfold)

# Print the average and standard deviation of the scores
print("Mean score: {:.2f}".format(scores.mean()))
print("Standard deviation: {:.2f}".format(scores.std()))

Mean score: 0.82
Standard deviation: 0.00


In [ ]:
# Split your data into features and target variable
X = ncaa_wmns_df2.drop('Win1_Lose0', axis=1)
y = ncaa_wmns_df2['Win1_Lose0']

model = xgb.XGBRegressor()

# define the cross-validation procedure
cv = KFold(n_splits=5, shuffle=True, random_state=123)

# fit the model using cross-validation
scores = xgb.cv(model.get_params(), xgb.DMatrix(X), nfold=5, metrics='rmse', seed=123)

# calculate the mean cross-validation score
mean_score = scores['test-rmse-mean'].mean()
print('Mean RMSE:', mean_score)

In [ ]:
ncaa_wmns_now_df = ncaa_wmns_df.loc[ncaa_wmns_df['Season'] == 2023]
ncaa_wmns_now_df

In [ ]:
# Create a list of all possible team matchups
team_ids = ncaa_wmns_now_df['TeamID'].unique()
team_matchups = list(itertools.product(team_ids, repeat=2))

# Create a new DataFrame with all possible team matchups
matchup_df = pd.DataFrame(team_matchups, columns=['Team1', 'Team2'])

# Make a copy of the matchup DataFrame and swap the Team1 and Team2 columns
reverse_matchup_df = matchup_df[['Team2', 'Team1']].rename(columns={'Team2': 'Team1', 'Team1': 'Team2'})

# Combine the original and reversed matchup DataFrames to get all possible matchups
all_matchups_df = pd.concat([matchup_df, reverse_matchup_df]).drop_duplicates().reset_index(drop=True)

# Use your XGBoost model to predict the winner of each matchup
xgb_predictions = xgb_model.predict(all_matchups_df)

# Create a new DataFrame with the predicted winners and matchup information
predictions_df = all_matchups_df.copy()
predictions_df['Winner'] = xgb_predictions

In [ ]:
import itertools

team1 = ncaa_wmns_now_df[['TeamID']].rename(columns={'TeamID': 'Team1'})
team2 = ncaa_wmns_now_df[['TeamID']].rename(columns={'TeamID': 'Team2'})

all_matchups_df = pd.DataFrame(list(itertools.product(team1['Team1'], team2['Team2'])), columns=['Team1', 'Team2'])

prediction_columns = ['Season', 'TeamID', 'Score', 'Seed', 'NumOT', 'WLoc', 'WFGM3', 'WFTM', 'WDR', 'WOR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM3', 'LFTM', 'LDR', 'LOR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']
ncaa_wmns_now_df = ncaa_wmns_now_df[prediction_columns]


# Merge the ncaa_wmns_df2 DataFrame with the all_matchups_df DataFrame on the TeamID column
all_matchups_df = pd.merge(all_matchups_df, ncaa_wmns_now_df, left_on='Team1', right_on='TeamID')
all_matchups_df = pd.merge(all_matchups_df, ncaa_wmns_now_df, left_on='Team2', right_on='TeamID', suffixes=('_1', '_2'))

# Drop the TeamID columns
all_matchups_df.drop(['TeamID_1', 'TeamID_2'], axis=1, inplace=True)

# Reorder the columns to match the order in the ncaa_wmns_df2 DataFrame
all_matchups_df = all_matchups_df[ncaa_wmns_now_df.columns[:-1]]


In [ ]:
import itertools

team1 = ncaa_wmns_now_df[['TeamID']].rename(columns={'TeamID': 'Team1'})
team2 = ncaa_wmns_now_df[['TeamID']].rename(columns={'TeamID': 'Team2'})

all_matchups_df = pd.DataFrame(list(itertools.product(team1['Team1'], team2['Team2'])), columns=['Team1', 'Team2'])

In [ ]:
prediction_columns = ['Season', 'TeamID', 'Score', 'Seed', 'NumOT', 'WLoc', 'WFGM3', 'WFTM', 'WDR', 'WOR',\
                      'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM3', 'LFTM', 'LDR', 'LOR', 'LAst', 'LTO',\
                      'LStl', 'LBlk', 'LPF']
ncaa_wmns_now_df = ncaa_wmns_now_df[prediction_columns]

In [ ]:
# Merge the ncaa_wmns_df2 DataFrame with the all_matchups_df DataFrame on the TeamID column
all_matchups_df = pd.merge(all_matchups_df, ncaa_wmns_now_df, left_on='Team1', right_on='TeamID')


In [ ]:
all_matchups_df = pd.merge(all_matchups_df, ncaa_wmns_now_df, left_on='Team2', right_on='TeamID', suffixes=('_1', '_2'))


In [ ]:
# Drop the TeamID columns
all_matchups_df.drop(['TeamID_1', 'TeamID_2'], axis=1, inplace=True)


In [ ]:
# Reorder the columns to match the order in the ncaa_wmns_df2 DataFrame
all_matchups_df = all_matchups_df[ncaa_wmns_now_df.columns[:-1]]

In [ ]:
# Define the column you want to generate all possible pairs for
column_name = 'TeamID'

# Get all unique values in the column
unique_values = ncaa_wmns_now_df[column_name].unique()

# Generate all possible pairs of values
value_pairs = list(itertools.combinations(unique_values, 2))

# Create a DataFrame to store the predictions for each pair
predictions_df = pd.DataFrame(columns=['Team' + '_1', 'Team' + '_2', 'Prediction'])

# Loop through each pair of values and make a prediction
for pair in value_pairs:
    # Create a new DataFrame containing the features for the matchup you want to predict
    matchup_features = pd.DataFrame({
        'GameID': [0, 0],
        'Season': [0, 0],
        'DayNum': [0, 0],
        'TeamID': [pair[0], pair[1]],
        'Seed': [0, 0],
        'Score': [0, 0],
        'WLoc': [0, 0],
        'NumOT': [0, 0],
        'WFGA': [0, 0],
        'WFGM3': [0, 0],
        'WFGA3': [0, 0],
        'WFTM': [0, 0],
        'WFTA': [0, 0],
        'WOR': [0, 0],
        'WDR': [0, 0],
        'WAst': [0, 0],
        'WTO': [0, 0],
        'WStl': [0, 0],
        'WBlk': [0, 0],
        'WPF': [0, 0],
        'LFGA': [0, 0],
        'LFGM3': [0, 0],
        'LFGA3': [0, 0],
        'LFTM': [0, 0],
        'LFTA': [0, 0],
        'LOR': [0, 0],
        'LDR': [0, 0],
        'LAst': [0, 0],
        'LTO': [0, 0],
        'LStl': [0, 0],
        'LBlk': [0, 0],
        'LPF': [0, 0],
    })

    # Replace prediction = xgb_model.predict(matchup_features) with the following line
    prediction = xgb_model.predict_proba(matchup_features)

    # Extract the probability of the positive class (i.e., team 1 wins)
    proba_team_1_wins = prediction[0, 1]

    # Append the prediction to the DataFrame
    predictions_df = predictions_df.append({
        'Team' + '_1': pair[0],
        'Team' + '_2': pair[1],
        'Prediction': proba_team_1_wins
    }, ignore_index=True)

# Print the predictions DataFrame
predictions_df

In [ ]:
# Define the column you want to generate all possible pairs for
column_name = 'TeamID'

# Get all unique values in the column
unique_values = ncaa_wmns_now_df[column_name].unique()

# Generate all possible pairs of values
value_pairs = list(itertools.combinations(unique_values, 2))

# Create a DataFrame to store the predictions for each pair
predictions_df = pd.DataFrame(columns=['Team' + '_1', 'Team' + '_2', 'Prediction'])

# Loop through each pair of values and make a prediction
for pair in value_pairs:
    # Create a new DataFrame containing the features for the matchup you want to predict
    matchup_features = pd.DataFrame({
        'GameID': [0, 1, 2, 3],
        'Season': [0, 0, 0, 0],
        'DayNum': [0, 0, 0, 0],
        'TeamID': [pair[0], pair[1], pair[0], pair[1]],
        'Seed': [0, 0, 0, 0],
        'Score': [0, 0, 0, 0],
        'WLoc': [0, 0, 0, 0],
        'NumOT': [0, 0, 0, 0],
        'WFGA': [0, 0, 0, 0],
        'WFGM3': [0, 0, 0, 0],
        'WFGA3': [0, 0, 0, 0],
        'WFTM': [0, 0, 0, 0],
        'WFTA': [0, 0, 0, 0],
        'WOR': [0, 0, 0, 0],
        'WDR': [0, 0, 0, 0],
        'WAst': [0, 0, 0, 0],
        'WTO': [0, 0, 0, 0],
        'WStl': [0, 0, 0, 0],
        'WBlk': [0, 0, 0, 0],
        'WPF': [0, 0, 0, 0],
        'LFGA': [0, 0, 0, 0],
        'LFGM3': [0, 0, 0, 0],
        'LFGA3': [0, 0, 0, 0],
        'LFTM': [0, 0, 0, 0],
        'LFTA': [0, 0, 0, 0],
        'LOR': [0, 0, 0, 0],
        'LDR': [0, 0, 0, 0],
        'LAst': [0, 0, 0, 0],
        'LTO': [0, 0, 0, 0],
        'LStl': [0, 0, 0, 0],
        'LBlk': [0, 0, 0, 0],
        'LPF': [0, 0, 0, 0],
    })

    # Replace prediction = xgb_model.predict(matchup_features) with the following line
    prediction = xgb_model.predict_proba(matchup_features)

    # Extract the probability of the positive class (i.e., team 1 wins)
    proba_team_1_wins = prediction[0, 1]

    # Append the prediction to the DataFrame
    predictions_df = predictions_df.append({
        'Team' + '_1': pair[0],
        'Team' + '_2': pair[1],
        'Prediction': proba_team_1_wins
    }, ignore_index=True)

# Print the predictions DataFrame
predictions_df

In [ ]:
predictions_df.value_counts('Prediction')

In [ ]:
# Define the column you want to generate all possible pairs for
column_name = 'TeamID'

# Get all unique values in the column
unique_values = ncaa_wmns_now_df[column_name].unique()

# Generate all possible pairs of values
value_pairs = list(itertools.combinations(unique_values, 2))

# Create a DataFrame to store the predictions for each pair
predictions_df2 = pd.DataFrame(columns=['Team' + '_1', 'Team' + '_2', 'Prediction'])

# Loop through each pair of values and make a prediction
for pair in value_pairs:
    # Create a new DataFrame containing the features for the matchup you want to predict
    matchup_features = pd.DataFrame({
        'GameID': [0, 1, 2, 3],
        'Season': [0, 0, 0, 0],
        'DayNum': [0, 0, 0, 0],
        'TeamID': [pair[0], pair[1], pair[0], pair[1]],
        'Seed': [0, 0, 0, 0],
        'Score': [0, 0, 0, 0],
        'WLoc': [0, 0, 0, 0],
        'NumOT': [0, 0, 0, 0],
        'WFGA': [0, 0, 0, 0],
        'WFGM3': [0, 0, 0, 0],
        'WFGA3': [0, 0, 0, 0],
        'WFTM': [0, 0, 0, 0],
        'WFTA': [0, 0, 0, 0],
        'WOR': [0, 0, 0, 0],
        'WDR': [0, 0, 0, 0],
        'WAst': [0, 0, 0, 0],
        'WTO': [0, 0, 0, 0],
        'WStl': [0, 0, 0, 0],
        'WBlk': [0, 0, 0, 0],
        'WPF': [0, 0, 0, 0],
        'LFGA': [0, 0, 0, 0],
        'LFGM3': [0, 0, 0, 0],
        'LFGA3': [0, 0, 0, 0],
        'LFTM': [0, 0, 0, 0],
        'LFTA': [0, 0, 0, 0],
        'LOR': [0, 0, 0, 0],
        'LDR': [0, 0, 0, 0],
        'LAst': [0, 0, 0, 0],
        'LTO': [0, 0, 0, 0],
        'LStl': [0, 0, 0, 0],
        'LBlk': [0, 0, 0, 0],
        'LPF': [0, 0, 0, 0],
    })
    # Make a prediction using your XGBoost model
    prediction = xgb_model.predict(matchup_features)

    # Append the prediction to the DataFrame
    predictions_df2 = predictions_df2.append({
        'Team' + '_1': pair[0],
        'Team' + '_2': pair[1],
        'Prediction': prediction[0]
    }, ignore_index=True)

# Print the predictions DataFrame
predictions_df2

In [ ]:
predictions_df2.value_counts('Prediction')

In [ ]:
games_df.groupby(['WTeamID', 'LTeamID']).mean()

In [ ]:
url = '/Users/aheinke/Documents/Flatiron/NYC-DS-010923/Phase_3/Phase3_Proj/PROJ_CSVs/Womens_NCAAW_Data_2023.html'  # Replace with the path to your HTML file
df = pd.read_html(url)[0]
df.to_csv('data.csv', index=False)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
games_df.groupby(['WTeamID', 'LTeamID', 'WLoc']).mean()

In [ ]:
teams_df = pd.read_csv('/Users/aheinke/Documents/Flatiron/NYC-DS-010923/Phase_3/Phase3_Proj/PROJ_CSVs/WTeamConferences.csv')
teams_df

In [ ]:
teams_now_df = teams_df[teams_df['Season'] == 2023]
teams_now_df

In [ ]:
import itertools
# create a list of all possible pairs of values
pairs = list(itertools.combinations(teams_now_df['TeamID'], 2))

# create a new dataframe with the pairs
pair_df = pd.DataFrame(pairs, columns=['Pair1', 'Pair2'])

# print the new dataframe
pair_df

In [ ]:
now_season_df = ncaa_wmns_df[ncaa_wmns_df['Season'] == 2023]
now_season_df

In [ ]:
X = ncaa_wmns_df2.drop(['Win1_Lose0', 'WFGM', 'LFGM'], axis=1)
y = ncaa_wmns_df2['Win1_Lose0']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

xgb_model = xgb.XGBClassifier()

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

# Prepare the new dataset
new_data = ncaa_wmns_now_df.drop(['Win1_Lose0', 'WFGM', 'LFGM'], axis=1)

new_data = new_data[X_train.columns]

# Scale the new data
X_new = scaler.transform(new_data)

# Predict outcomes
y_pred2 = xgb_model.predict(X_new)